In [18]:
import re
import os
import subprocess
source = "COGENT Public Looking Glass"
# destination = ["google.txt","apple.txt","bitcoin.txt","goldmansachs.txt","ntt.txt","spotify.txt","vodacom.txt","wikipedia.txt","oracle.txt"]
# destination = ["google.txt","apple.txt","bitcoin.txt","ntt.txt","spotify.txt"]
# destination = ["google.txt","alibaba.txt","bitcoin.txt","ntt.txt","spotify.txt","amazon.txt"]
destination = ["spotify.txt"]
def getdata(source,destination):
    file = open(source+"/"+destination,"r")
    data = {
        "AS NUMBER" : [],
        "IP ADDRESS": [],
        "IP RANGE" : [],
        "LOCATION" : [],
        "ORGANISATION": []
    }
    for line in file.readlines():

    
        s = re.findall('\[AS[0-9]+',line)
        
        if (len(s)>0 and s[0][1] != "*"):
            
            ips = re.findall('[0-9]+\.[0-9]+\.[0-9]+\.[0-9]+',line)

            statement = "./IPGeoLocation/ipgeolocation.py -t {} ".format(ips[0])
            proc = subprocess.Popen([statement],stdout=subprocess.PIPE,shell=True)
            (out,err) = proc.communicate()
            out = out.decode()
            city = re.findall('City:[^\n]*\\n',out)
            city = re.findall('[A-Z][a-zA-Z ]*',city[0])
            country= re.findall('Country:[^\n]*\\n',out)
            country = re.findall('[A-Z][a-zA-Z ]*',country[0])
            org= re.findall('Organization:[^\n]*\\n',out)
            org = re.findall('[A-Z][a-zA-Z ]*',org[0])
            if (len(org) >=2):
                org = org[1]
            else:
                org = ""
            if (len(city) >=2):
                city = city[1]
            else:
                city = ""
            if (len(country) >=2):
                country = country[1]
            else:
                country = ""
            
            loc = city+'/'+country
            statement = "whois -a {}".format(ips[0])
            proc2 = subprocess.Popen([statement],stdout=subprocess.PIPE,shell=True)
            (out,err) = proc2.communicate()
            out = out.decode()
            iprange = re.findall('inetnum:[ \t]*[-0-9\. ]*|NetRange:[ \t]*[-0-9\. ]*',out)
            if (len(iprange) > 0):
                iprange = iprange[0].split(":")[1].lstrip()
            else:
                iprange = ""
                
            
            data["IP RANGE"].append(iprange)
            data["AS NUMBER"].append(s[0][1:])
            data["IP ADDRESS"].append(ips[0])
            data["ORGANISATION"].append(org)
            data["LOCATION"].append(loc)
            
    file.close()
    return data


In [19]:
# getdata(source,destination[0])
destination_dict = {}
for d in destination:
    print(d)
    destination_dict[d] = getdata(source,d)
destination_dict

spotify.txt


{'spotify.txt': {'AS NUMBER': ['AS174', 'AS15169', 'AS396982'],
  'IP ADDRESS': ['130.117.254.105', '72.14.196.162', '25.224.186.35'],
  'IP RANGE': ['', '72.14.192.0 - 72.14.255.255', '25.0.0.0 - 25.255.255.255'],
  'LOCATION': ['Amsterdam/Netherlands',
   'Mountain View/United States',
   'London/United Kingdom'],
  'ORGANISATION': ['PSINet', 'Google LLC', 'UK Ministry of Defence']}}

In [20]:
destination_dict

{'spotify.txt': {'AS NUMBER': ['AS174', 'AS15169', 'AS396982'],
  'IP ADDRESS': ['130.117.254.105', '72.14.196.162', '25.224.186.35'],
  'IP RANGE': ['', '72.14.192.0 - 72.14.255.255', '25.0.0.0 - 25.255.255.255'],
  'LOCATION': ['Amsterdam/Netherlands',
   'Mountain View/United States',
   'London/United Kingdom'],
  'ORGANISATION': ['PSINet', 'Google LLC', 'UK Ministry of Defence']}}

In [17]:
import pandas as pd

for d in destination:
    df = pd.DataFrame(destination_dict[d])
    df.to_excel(source +"/"+"structured" + "/" + d[:-3]+"xlsx")